In [2]:
%%time
from twython import Twython
from twython import TwythonError, TwythonRateLimitError, TwythonAuthError
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time
import datetime
import csv
#import nltk
#import nltk.tag.stanford as st
import ner
import pandas as pd
import numpy as np
import re
from pandas import ExcelWriter
from collections import defaultdict
from fuzzywuzzy import fuzz,process
import unicodecsv

CPU times: user 541 ms, sys: 603 ms, total: 1.14 s
Wall time: 1.66 s


/Users/jackiemartinez/anaconda/lib/python2.7/site-packages/fuzzywuzzy/fuzz.py:33: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
CONSUMER_KEY = 'VEW2a7ttqT7rUrfi88ql02rpd'
CONSUMER_SECRET ='susbqqPl1WU3C8d4jGgxaCUBebmYuPRXljwQr4N858FXyVCMFj'
OAUTH_TOKEN = '27459655-dGoeUZP6x92W7C97ZBp2kuZWnnFBDBfRmJgtD9fwS'
OAUTH_TOKEN_SECRET = 'nP7CfdiiIhvjzUlUg38qgVoR5nWjWqpv8NaIcxpMzY8Yp'

#auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,CONSUMER_KEY, CONSUMER_SECRET)
api = Twython(CONSUMER_KEY, CONSUMER_SECRET, OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

In [5]:
full_tweet_css = ".js-stream-item.stream-item.stream-item.expanding-stream-item"
tweet_text_css = ".TweetTextSize.js-tweet-text.tweet-text"
timestamp_css = '.tweet-timestamp.js-permalink.js-nav.js-tooltip'
quote_tweet_css = ".QuoteTweet-text.tweet-text.u-dir"
rt_css = '.ProfileTweet-action--retweet.u-hiddenVisually'
fav_css = ".ProfileTweet-action--favorite.u-hiddenVisually"
action_css = '.ProfileTweet-actionCount'

In [3]:
team_list_full = ["DC Breeze","Cincinnati Revolution","Chicago Fire","Charlotte Express",
             "Atlanta Hustle","Detroit Mechanix","INDIANAPOLIS ALLEYCATS","Jacksonville Cannons",
             "LOS ANGELES AVIATORS","MADISON RADICALS","MINNESOTA WIND CHILL","MONTREAL ROYAL",
             "NASHVILLE NIGHTWATCH","New York Empire","Ottawa Outlaws","PHILADELPHIA PHOENIX",
             "PITTSBURGH THUNDERBIRDS","Raleigh Flyers","Rochester Dragons","SAN DIEGO GROWLERS",
             "SAN FRANCISCO FLAMETHROWERS","SAN JOSE SPIDERS","SEATTLE CASCADES","TORONTO RUSH",
             "VANCOUVER RIPTIDE"]
team_list_full = [t.lower() for t in team_list_full]
team_cities = ["dc","cincinnati","chicago","charlotte","atlanta","detroit","indianapolis",
               "jacksonville","los angeles","madison","minnesota","montreal","nashville",
               "new york","ottawa","philadelphia","pittsburgh","raleigh","rochester",
               "san diego","san francisco","san jose","seattle","toronto","vancouver"]
team_names = ['breeze','revolution','fire','express','hustle','mechanix',
              'alleycats','cannons','aviators','radicals','wind chill','royal',
              'nightwatch','empire','outlaws','phoenix','thunderbirds','flyers',
              'dragons','growlers','flamethrowers','spiders','cascades','rush','riptide']
alt_name = ["dc","cincy","chicago","charlotte","atlanta","detroit","indy",
            "jax","la","madison","minnesota","montreal","nashville",
            "ny","ottawa","philly","pittsburgh","raleigh","rochester",
            "sd","sf","sj","seattle","toronto","vancouver"]
team_dict = dict(zip(team_cities, zip(team_names,alt_name)))


In [78]:
for k,v in team_dict.iteritems():
    print k,v

charlotte ('express', 'charlotte')
madison ('radicals', 'madison')
cincinnati ('revolution', 'cincy')
san jose ('spiders', 'sj')
minnesota ('wind chill', 'minnesota')
ottawa ('outlaws', 'ottawa')
san francisco ('flamethrowers', 'sf')
seattle ('cascades', 'seattle')
rochester ('dragons', 'rochester')
chicago ('fire', 'chicago')
san diego ('growlers', 'sd')
philadelphia ('phoenix', 'philly')
new york ('empire', 'ny')
vancouver ('riptide', 'vancouver')
indianapolis ('alleycats', 'indy')
pittsburgh ('thunderbirds', 'pittsburgh')
nashville ('nightwatch', 'nashville')
dc ('breeze', 'dc')
atlanta ('hustle', 'atlanta')
jacksonville ('cannons', 'jax')
toronto ('rush', 'toronto')
detroit ('mechanix', 'detroit')
raleigh ('flyers', 'raleigh')
los angeles ('aviators', 'la')
montreal ('royal', 'montreal')


In [3]:
def get_tweets(query_str, min_unixtime):
    tweets = []
    driver = webdriver.Chrome()
    driver.set_window_size(500, 500)
    #pt2 = "%20until%3A2015-04-05&src=typd"
    url_to_get = "https://twitter.com/search?q=" + query_str #+ pt2
    print "URL TO GET: ", url_to_get
    #url_to_get = "https://twitter.com/search?q=" + query_str
    driver.get(url_to_get)
    
    while True:
        index = len(tweets)
        tweets_whole = driver.find_elements_by_css_selector(full_tweet_css)[index:]
        for elt in tweets_whole:
            # get tweet text
            try:
                t_text = elt.find_element_by_css_selector(tweet_text_css).text
            except NoSuchElementException:
                print "WARNING: No Text Element around: ", index
                t_text = "ERROR: no text element found"
            # get tweet timestamp
            try:
                t_unixtime = (int(elt.find_element_by_css_selector(timestamp_css)
                          .find_element_by_css_selector("span")
                          .get_attribute("data-time")))
            except NoSuchElementException:
                print "WARNING: No Time Element around: ", index
                #t_unixtime = elt.get_attribute('innerHTML')
                # defaults to 1/1/2016 (impossible, so it will show up as a red flag in the data)
                t_unixtime = 1451606400
            # get any quoted tweets within the tweet
            try:
                t_quote_text = elt.find_element_by_css_selector('.QuoteTweet-text.tweet-text.u-dir').text
                quote_tup = (t_quote_text, t_unixtime, 0)
                tweets.append(quote_tup)
            except NoSuchElementException:
                pass
            # get sum of RTs and favorites
            try:
                rts = (elt
                       .find_element_by_css_selector(rt_css)
                       .find_element_by_css_selector(action_css)
                       .get_attribute("data-tweet-stat-count"))
                favs = (elt
                       .find_element_by_css_selector(fav_css)
                       .find_element_by_css_selector(action_css)
                       .get_attribute("data-tweet-stat-count"))
                rts_plus_favs = int(rts) + int(favs)
            except NoSuchElementException:
                rts_plus_favs = 0
            tweet_tup = (t_text,t_unixtime,rts_plus_favs)
            tweets.append(tweet_tup)
        if int(tweets[-1][1]) < min_unixtime:
            print "FINISHED, WITH LAST TWEET AT: ", tweets[-1][1]
            driver.quit()
            return(tweets)
        print "Most Recent Unix Time: ", int(tweets[-1][1])
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
    driver.quit()

In [6]:
%%time
#tweets = get_tweets("audl", 1420070400)
tweet_grab = get_tweets("audl", 1420070400)

URL TO GET:  https://twitter.com/search?q=audl
Most Recent Unix Time:  1448982374
Most Recent Unix Time:  1448459812
Most Recent Unix Time:  1447700045
Most Recent Unix Time:  1447118607
Most Recent Unix Time:  1446677072
Most Recent Unix Time:  1446278132
Most Recent Unix Time:  1445969764
Most Recent Unix Time:  1444941006
Most Recent Unix Time:  1444351291
Most Recent Unix Time:  1444236727
Most Recent Unix Time:  1443645603
Most Recent Unix Time:  1443112062
Most Recent Unix Time:  1442591581
Most Recent Unix Time:  1442251047
Most Recent Unix Time:  1441740726
Most Recent Unix Time:  1441123319
Most Recent Unix Time:  1440874246
Most Recent Unix Time:  1440771014
Most Recent Unix Time:  1440628329
Most Recent Unix Time:  1440212327
Most Recent Unix Time:  1439959783
Most Recent Unix Time:  1439590362
Most Recent Unix Time:  1439333131
Most Recent Unix Time:  1439214343
Most Recent Unix Time:  1439162851
Most Recent Unix Time:  1439161113
Most Recent Unix Time:  1439152667
Most Rec

In [7]:
print len(tweet_grab)

2120


In [12]:
## Write to file
# with open('audl_tweets.csv','w') as out:
#     csv_out=csv.writer(out)
#     csv_out.writerow(['tweet_text','tweet_unixtime', 'tweet_popularity'])
#     for row in tweet_grab:
#         new_tup = (row[0].encode("utf8"), row[1], row[2])
#         csv_out.writerow(new_tup)

In [18]:
df = pd.read_excel('audl_tweets.xls')

In [19]:
df.shape

(2085, 3)

In [20]:
df.head()

,tweet_text,tweet_unixtime,tweet_popularity
0,Recruit Camp #1 tonight! http://cincinnatirevo...,1449359003,8
1,"Sin The Fields: Rochester Dragons, AUDL Advice...",1449262946,0
2,New STF! This time we tried the strategy of no...,1449258052,1
3,".@SinTheFields: Rochester Dragons, @theAUDL Ad...",1449257125,4
4,We're making the storyboard for the 2016 #AUDL...,1449180084,9


In [21]:
#take away new lines
df['tweet_text'] = df['tweet_text'].apply(lambda x: re.sub(r'\n',' ',x))

In [22]:
for elt in df.tweet_text:
    print type(elt
    print "--"

Recruit Camp #1 tonight! http://cincinnatirevolution.com/2016-revolution-tryouts/ â€¦ #AUDL #cincy #ultimate 
--
Sin The Fields: Rochester Dragons, AUDL Advice, & Reader Mail http://ow.ly/38Gt7K 
--
New STF! This time we tried the strategy of not releasing it 10 mins before a global tragedy! (finger's crossed) http://ultiworld.com/2015/12/04/sin-the-fields-rochester-dragons-audl-advice-reader-mail/ â€¦ â€¦
--
.@SinTheFields: Rochester Dragons, @theAUDL Advice, & Reader Mail: http://ultiworld.com/2015/12/04/sin-the-fields-rochester-dragons-audl-advice-reader-mail/ â€¦
--
We're making the storyboard for the 2016 #AUDL mag. What do you want to see in it? 
--
What a pro looks like. #AUDL #SOTG 
--
#ultimate AUDL November News Roundup - There has been a lot going on in the offseason for the American Ultimate... http://ow.ly/38DDGE 
--
.@theAUDL November News Roundup: http://ultiworld.com/2015/12/03/audl-november-news-roundup/ â€¦
--
Today is International Spirit of the Game Day: Celebrate t

In [124]:
## Extract user mentions
def get_username_set(all_tweets_text):
    username_set = []
    for tweet_text in all_tweets_text:
        users = re.findall(r'@(\w+)', tweet_text)
        username_set.extend(users)
    #user_set = set(user_set)
    return set(username_set)
## Finds users' names based on list of usernames -- USES TWITTER API!!!
def get_names(usernames,twitter_instance):
    user_dict = {}
    indexes = range(0,len(usernames),100)
    indexes.append(len(usernames) - 1)
    for i in range(len(indexes) - 1):
        u_list = usernames[indexes[i]:indexes[i+1]]
        print (indexes[i]),(indexes[i+1])
        u_csv = ','.join(u_list)
        #print u_csv
        res_dicts = twitter_instance.lookup_user(screen_name = u_csv)
        for elt in res_dicts:
            user_dict[elt['screen_name'].lower()] =elt['name'].title()
    return user_dict



In [125]:
uname_set = get_username_set(df.tweet_text)

In [126]:
list(uname_set)[len(uname_set) - 1]

u'ditcheverything'

In [127]:
username_dict = get_names(list(uname_set),api)

0 100
100 200
200 300
300 400
400 441


In [128]:
len(username_dict.keys())

409

In [139]:
# Write Dictionary to File
w = csv.writer(open("username_dict.csv", "w"))
for key, val in username_dict.items():
    w.writerow([key, val.encode('utf-8')])

In [130]:
def replace_username(old_tweet,name_dict):
    for user in re.findall(r'@(\w+)', old_tweet):
        try:
            nm = name_dict[user.lower()]
        except KeyError:
            print "User not found: ", user
            nm = user
        old_tweet = re.sub(user,nm,old_tweet)
    nt = re.sub(r'@','',old_tweet)
    return(nt)

In [132]:
df['tweet_text'] = df['tweet_text'].apply(lambda x: replace_username(x,username_dict))

User not found:  ditcheverything
User not found:  GDPJ_Meyer
User not found:  CincySaints
User not found:  charlotteexpress
User not found:  56ersNPSL
User not found:  charlotteaudl
User not found:  Bolton_Lava
User not found:  CharlotteAUDL


In [146]:
## Save work up to this point:
writer = ExcelWriter('audl_df.xlsx')
df.to_excel(writer,'Sheet1')
writer.save()

##Open Files from Directory
df: audl_df.xlsx

dictionary: username_dict.csv

In [4]:
df = pd.read_excel('audl_df.xlsx')
username_dict = {}
reader = csv.reader(open("username_dict.csv"))
for row in reader:
    username_dict[row[0]]=row[1:]

In [5]:
# s2 = 'Sin The Fields: Rochester Dragons,  Advice, & Reader Mail http://ow.ly/38Gt7K '
# clean_tweets_2(s2)

In [6]:
# s3 = df.tweet_text[1148]
# print s3
# clean_tweets_2(s3)

In [7]:
not_names = ['audl','ultiworld','mlu','sotg']
def clean_tweets_2(t):
    potential_teams = []
    pat = ('|'.join(not_names))
    t = re.sub('(?i)'+pat,'',t)
    t = re.sub('#','',t)
    for k,v in team_dict.iteritems():
        if k in t.lower() or v[0] in t.lower() or (' ' + v[1] + ' ') in t.lower():
            pat = '|'.join([k,v[0],(' ' + v[1] + ' ')])
            t = re.sub('(?i)' + pat,'',t)
            potential_teams.append(k)
    return (potential_teams,t)

In [8]:
res_clean = df['tweet_text'].apply(clean_tweets_2)
df['clean'] = [twt for (team,twt) in res_clean]
df['poss_team'] = [team for (team,twt) in res_clean]


In [9]:
df.head()

,tweet_text,tweet_unixtime,tweet_popularity,clean,poss_team
0,Recruit Camp #1 tonight! http://cincinnatirevo...,1449359003,8,Recruit Camp 1 tonight! http://.com/2016--tryo...,[cincinnati]
1,"Sin The Fields: Rochester Dragons, AUDL Advice...",1449262946,0,"Sin The Fields:, Advice, & Reader Mail http:/...",[rochester]
2,New STF! This time we tried the strategy of no...,1449258052,1,New STF! This time we tried the strategy of no...,[rochester]
3,".Sinthefields: Rochester Dragons, Audl Advice,...",1449257125,4,".Sinthefields:, Advice, & Reader Mail: http:/...",[rochester]
4,We're making the storyboard for the 2016 #AUDL...,1449180084,9,We're making the storyboard for the 2016 mag....,[]


In [10]:
# Run in different terminal before executing code below
# java -mx1000m -cp stanford-ner.jar edu.stanford.nlp.ie.NERServer \
#     -loadClassifier classifiers/english.all.3class.caseless.distsim.crf.ser.gz \
#     -port 8080 -outputFormat inlineXML
tagger = ner.SocketNER(host='localhost', port=8080)

In [11]:
def process_tweet(twt,pop,poss_team,dpop,dteam):
    try:
        persons = tagger.get_entities(twt)['PERSON']
    except KeyError:
        persons = []
    for p in persons:
        if len(p.split()) == 4:
            if p.split()[0] == p.split()[2] and p.split()[1]==p.split()[3]:
                p = ' '.join(p.split()[0:2])
        dpop[p.lower()] += pop + 1
        dteam[p.lower()].extend(poss_team)

In [12]:
name_dict =  defaultdict(int)
poss_team_dict = defaultdict(list)
#apply function over dataframe to modify dictionary
dummy_var = df.apply(lambda r: process_tweet(r.clean, r.tweet_popularity, r.poss_team
                                             ,name_dict, poss_team_dict)
                     ,axis=1)


In [13]:
sorted(name_dict.items(), key=lambda x:x[1], reverse=True)

[(u'brodie smith', 190),
 (u'beau kittredge', 159),
 (u'evan lepler', 157),
 (u'silverman', 90),
 (u'audrey larson', 85),
 (u'jonathan helton', 81),
 (u'matt smith', 80),
 (u'icymi', 76),
 (u'silverman espnassignmentdesk', 74),
 (u'jimmy mickle', 74),
 (u'tyler degirolamo', 62),
 (u'justin allen', 59),
 (u'cameron brock', 57),
 (u'ultimatefrisbee  gopro', 53),
 (u'andrew meshnick', 51),
 (u'sean mccall', 51),
 (u'ed fox', 51),
 (u'ty', 48),
 (u'colin grandon', 47),
 (u'husayn carnegie', 44),
 (u'callahan', 39),
 (u'pat shriwise', 38),
 (u'plew', 36),
 (u'gordon', 36),
 (u'nick hutton', 35),
 (u'jay clark', 32),
 (u'nick lance', 32),
 (u'ryan simmons', 32),
 (u'rob lloyd', 32),
 (u'sean mott', 31),
 (u'steve gordon', 30),
 (u'suivez  demain', 30),
 (u'alex thorne pgh', 28),
 (u'darren wu', 28),
 (u'ashlin joye', 27),
 (u'formerbitmap cutter anthony mclean', 26),
 (u'nick bowers', 25),
 (u'cohen', 25),
 (u'john cecil', 25),
 (u'jeff lindquist', 24),
 (u'janssen', 23),
 (u'colin', 22),
 (

In [14]:
poss_team_dict['tim']

['cincinnati', 'cincinnati']

In [15]:
roster_dict = {}
reader = csv.reader(open("combined_dict.csv"))
for row in reader:
    roster_dict[row[0]]=row[1:]

In [25]:
def unicode_csv_reader(utf8_data, dialect=csv.excel, **kwargs):
    csv_reader = csv.reader(utf8_data, dialect=dialect, **kwargs)
    for row in csv_reader:
        yield [unicode(cell, 'utf-8') for cell in row]

filename = 'combined_dict.csv'
reader = unicode_csv_reader(open(filename))

In [26]:
for row in reader:
    roster_dict[row[0]]=row[1:]

In [27]:
v = [v[0] for v in roster_dict.values()]
roster_dict2 = dict(zip(v,roster_dict.keys()))
#roster_dict['Frédérick Parenteau']

In [31]:
roster_dict2[u'André Arsenault']

u'Arsene A'

In [52]:
poss_team_dict['william rains']

[]

In [67]:
sorted(name_dict.keys())

[u'- sockeye vs denver johnny bravo np',
 u'a.j. nelson',
 u'aaron roberts',
 u'adam hills',
 u'adam nelson',
 u'aj jacoski',
 u'aj nelson',
 u'ajjaajjaajajaj adem\xe3',
 u'alan kolick',
 u'alex bill yeah',
 u'alex jacoski',
 u'alex korb',
 u'alex nord',
 u'alex thorne',
 u'alex thorne pgh',
 u'allaboutu  westerndivisionchamps',
 u'allan lamport',
 u'allstaru',
 u'amanda bootes whyi',
 u'andrea campagna ciao',
 u'andrew allen',
 u'andrew meshnick',
 u'andrew roca',
 u'andrew zill',
 u'andy barnhart',
 u'andy curtis cleanin',
 u'andy stevens',
 u'anna',
 u'anthony',
 u'anthony mclean',
 u'antlersup',
 u'antoine genest',
 u'ashlin',
 u'ashlin joye',
 u'atl  weektwo',
 u'atta',
 u'atthebuzzer odell beckham jrwild',
 u'audrey larson',
 u'audrey larson audrey larsonfinals',
 u'audrey larson audrey larsontv micah hood',
 u'audrey larson evan lepler',
 u'audrey larson lang syne',
 u'audrey larson west',
 u'audrey larson2014',
 u'austin sol',
 u'avi  espn3 shawn mccarthy',
 u'bama',
 u'bandos'

In [80]:
len(roster_dict2.keys())

799

In [70]:
choices = [k.lower() for k in (roster_dict2.keys())]
p = 'xtehn'
process.extract(p, choices, limit=10)

[('h', 90),
 ('x', 90),
 ('n', 90),
 ('t', 90),
 (u'frank wooten', 60),
 (u'jonathan palumbo', 54),
 (u'brian rathunde ', 54),
 (u'jonathan martin', 54),
 (u'andrew ouchtecheny', 54),
 (u'nathan bridges', 54)]

In [73]:
one_name = {}
two_name = defaultdict(int)
n_name = {}
choices = [k.lower() for k in (roster_dict2.keys())]

for k,v in name_dict.iteritems():
    if len(k.split()) == 2:
        match = process.extract(k,choices,limit=2)
        if match[0][1]>95:
            two_name[match[0][0]] += v
#     elif len(k.split()) % 2 == 0 and len(k.split()) > 2:
#         interval = range(0,len(k),2)
#         for i,intval in enumerate(interval):
            
#         two_name[k] = v
#     elif len(k.split()) > 2:
#         n_name[k] = v
#     else:
#         print "Error"

In [77]:
print len(two_name)
sorted(two_name.items(), key=lambda x:x[1], reverse=True)[0:10]

110


[(u'brodie smith   ', 190),
 (u'beau kittredge', 159),
 (u'matt smith', 80),
 (u'jimmy mickle', 74),
 (u'tyler degirolamo', 62),
 (u'justin allen', 59),
 (u'ed fox', 51),
 (u'sean mccall', 51),
 (u'andrew meshnick', 51),
 (u'colin grandon', 47)]

In [236]:
# for k,v in two_name.iteritems():
#     try:
#         print k,": ", poss_team_dict[k]
#     except KeyError:
#         pass

In [78]:
# choices = ['belu mason','belu kittredge','justin allen']
# process.extract('belu Km', choices)
# # # matched_dict = defaultdict(int)

# # #for names of length 2, simple match
# # for name, point in two_name.iter_items():
# #     choices = karine_dict.keys()
# #     match = process.extract(name, choices, limit = 3)
# #     # if not exact match, use other criteria
# #     if match[0][0] < 100:
        
# #     matched_dict[process.extract(name, choices, limit = 1)] += point

# # #for names of length 1, match with possible teams first and then with 

In [79]:
# tweets[2][0].encode("utf8").replace("\xe2\x80\xa6","...")